In [ ]:
import numpy as np
import tensorflow as tf
import random, math
import sktensor

In [ ]:
## Hyper-Parameters
n = 1000 # number of samples
d = 10  # input dimension
k = 5   # hidden layer size
kappa = 2 
T = 100 # num of epochs
lr = 0.02
batch_size = 20

In [ ]:
## Activation Function
def phi(h):
    return h**2 if h > 0 else 0
phi = np.vectorize(phi)

In [ ]:
## Data and Params Generation

gauss_mat = np.random.normal(0.0, 0.1 , (d,k))
U, V = np.linalg.qr(gauss_mat) 

### TODO: U,V both are orthogonal and come from different gaussian matrices

diag = []
v_gt = []
v_choice = [1,-1]
for iter in range(k):
    diag.append(1+1.*iter*(kappa-1)/(k-1))
    v_gt.append(random.choice(v_choice))
    
Sigma = np.diag(diag)
W_gt = np.dot(np.dot(U, Sigma), np.transpose(V))
v_gt = np.asarray(v_gt)
train_x = []
train_y = []
for iter in range(n):
    train_x.append(np.random.normal(0.0,1.,d))
    train_y.append(np.dot(phi(np.dot(train_x[iter], W_gt)),v_gt))
train_x = np.asarray(train_x)
train_y = np.transpose(np.asarray(train_y))

In [ ]:
def outer3(a,b,c):
    A = np.outer(a,b)
    B = []
    for third in c:
        B.append(A*third)
    return np.asarray(B)

def outer4(a,b,c,d):
    A = outer3(a,b,c)
    B = []
    for fourth in d:
        B.append(A*fourth)
    return np.asarray(B)

def outer3I(x):
    return outer3(x,x,x)

def outer4I(x):
    return outer4(x,x,x,x)

def specOuterI(x):
    d = len(x)
    iden = np.identity(d)
    final = np.zeros([d,d,d])
    for i in range(d):
        final += outer3(x, iden[i], iden[i]) + outer3(iden[i], x, iden[i])+ outer3(iden[i], iden[i],x)
    return final

def specOuterMat(M):
    d = np.shape(M)[0]
    ## TODO
    return np.zeros([d,d,d,d])
    
    

In [ ]:
def multLnr(M, argList):
    if len(np.shape(M)) == 3:
        a,b,c = argList

        assert(np.shape(M)[0] == np.shape(a)[0])
        assert(np.shape(M)[1] == np.shape(b)[0])
        assert(np.shape(M)[2] == np.shape(c)[0])
        ## HardCoding Here :: TODO
        res = np.zeros([np.shape(a)[-1],np.shape(b)[-1] ])
        for itera in range(np.shape(a)[0]):
            for iterb in range(np.shape(b)[0]):
                for iterc in range(np.shape(c)[0]):
                    res += M[itera, iterb, iterc]*c[iterc]*np.outer(a[itera], b[iterb])
        return res
    else:
        assert(len(np.shape(M)) == 4)
        a,b,c,d = argList
        if len(np.shape(c)) == 2:
            pass
        else:
            res = np.zeros([np.shape(a)[-1],np.shape(b)[-1] ])
            for itera in range(np.shape(a)[0]):
                for iterb in range(np.shape(b)[0]):
                    for iterc in range(np.shape(c)[0]):
                        for iterd in range(np.shape(d)[0]):
                            res += M[itera, iterb, iterc]*c[iterc]*d[iterd]*np.outer(a[itera], b[iterb])
            return res
        
def multLnr1(M, V):
    d = np.shape(M)[0]
    k = np.shape(V)[1]
    
    res = np.zeros((k,k))
    
    for itera in range(d):
        for iterb in range(d):
            res += M[itera, iterb]*np.outer(V[itera], V[iterb])
            
    return res

def multLnr2(P3, V):
    d = np.shape(P3)[0]
    k = np.shape(V)[1]
    
    res = np.zeros((k,k,k))
    
    for itera in range(d):
        for iterb in range(d):
            for iterc in range(d):
                res += P3[itera, iterb, iterc]*outer3(V[itera], V[iterb], V[iterc])
    
    return res

In [ ]:
# def prob(x):
#     return math.pow(math.e, -0.5*np.dot(np.transpose(x),x))/np.sqrt(math.pow(2*math.pi, len(x)))

def getM1(X, y):
    M1 = np.zeros(d)
    for iter in range(np.shape(X)[0]):
        M1 += y[iter]*X[iter]
    return M1/len(y)

def getM2(X, y):
    M2 = np.zeros([d,d])
    for iter in range(np.shape(X)[0]):
        M2 += y[iter]*(np.outer(X[iter], X[iter]) - np.identity(d))
    return M2/len(y)

def getM3(X, y):
    M3 = np.zeros([d,d,d])
    for iter in range(np.shape(X)[0]):
        M3 += y[iter]*(outer3I(X[iter]) - specOuterI(X[iter]) )
    return M3/len(y)

def getM4(X, y):
    M4 = np.zeros([d,d,d,d])
    for iter in range(np.shape(X)[0]):
        M4 += y[iter]*(outer4I(X[iter]) - specOuterMat(np.outer(X[iter], X[iter])) + specOuterMat(np.identity(d)) )
    return M4/len(y)

In [ ]:
def getP2V(V, X, y, k):
    d = np.shape(X)[1]
    P2V = np.zeros((d,k))
    for i in range(len(X)):
        P2V += y[i]*(np.dot(np.transpose([X[i]]), np.dot([X[i]], V) ) - V)
    return P2V/np.shape(X)[0]
        
def getP2v(v, X, y):
    d = np.shape(X)[1]
    P2v = np.zeros((d,1))
    for i in range(len(X)):
        P2v += y[i]*(np.transpose([X[i]])*np.dot([X[i]], np.transpose([v]))  - np.transpose([v]))
    return P2v/np.shape(X)[0]

def topk(eigenV, k):
    sortList = []
    for i in range(2):
        for j in range(k):
            sortList.append([eigenV[i,j], (i,j)])
    sortList.sort(reverse=True)
    k1 = 0
    k2 = 0
    pi1 = {}
    pi2 = {}
    for i in range(k):
        if sortList[i][1][0] == 0:
            pi1[k1] = sortList[i][1][1]
            k1 += 1
        else:
            pi2[k2] = sortList[i][1][1]
            k2 += 1
    return pi1, pi2, k1, k2
    

In [ ]:
def powMeth( k, X, y):
#     C = 3*np.linalg.norm(P2)
    C = 5
    T = 10
    d =  np.shape(X)[1]
    V1 = np.random.normal(0.0, 0.1 , (d,k))
    V2 = np.random.normal(0.0, 0.1 , (d,k))
    
    for i in range(T):
        P2V1 = getP2V(V1, X, y, k)
        P2V2 = getP2V(V2, X, y, k)
        V1, temp = np.linalg.qr(C*V1 + P2V1)
        V2, temp = np.linalg.qr(C*V2 - P2V2)
    
    eigenV = np.zeros((2,k))
    for i in range(k):
        eigenV[0,i] = abs( np.dot(np.transpose(V1[:,i]) , getP2v(V1[:,i], X, y) ) )
    for i in range(k):
        eigenV[1,i] = abs(np.dot(np.transpose(V2[:,i]) , getP2v(V2[:,i], X, y) ))
    
    pi1, pi2, k1, k2 = topk(eigenV, k)
    
    V1_new = np.zeros((d,k1))
    V2_new = np.zeros((d,k2))
    
    for i in range(k1):
        V1_new[:,i] = V1[:,pi1[i]]
    for i in range(k2):
        V2_new[:,i] = V2[:,pi2[i]]
    V2_new, temp = np.linalg.qr(np.dot(np.identity(d)-np.dot(V1_new, np.transpose(V1_new)), V2_new))
    return np.concatenate((V1_new, V2_new), axis=1)
    

In [ ]:
def recmagsgn(V, U, X, y):
    
    ### l1 and l2 :: TODO
    l1 = 1
    l2 = 2
    
    d = np.shape(X)[1]
    k = np.shape(U)[0]
    divInd = int(len(X)/2)
    
    # Partition
    X1 = X[:divInd]
    y1 = y[:divInd]

    X2 = X[divInd:]
    y2 = y[divInd:]
    
    alpha = np.random.normal(0.0, 0.1 , d)
    
    ## TODO: Assuming l1=1 and l2=2 
    Q1 = getM1(X1, y1)
    Q2 = multLnr1(getM2(X2,y2), V).flatten()    
    
    Vu = []
    UU = []

    for ind in range(len(U)):
        Vu.append(np.dot(V, U[ind]))
        UU.append(np.dot( np.transpose([U[ind]]), [U[ind]] ).flatten())
        
    ## Estimating z
    z_old = np.zeros(k)
    z_new = np.zeros(k)
    
    T = 1000
    for iterT in range(T):
        for ind in range(k):
            
            mult_fact = np.zeros(d)
            for j in range(k):
                if j != ind:
                    mult_fact += z_old[j]*Vu[j]
            div_fact = np.dot(np.transpose(Vu[ind]), Vu[ind])
            z_new[ind] = (np.dot(np.transpose(Q1), Vu[ind]) + np.dot(np.transpose(Vu[ind]), Q1) -
                                 np.dot(np.transpose(Vu[ind]), mult_fact) - 
                                  np.dot(np.transpose(mult_fact), Vu[ind]) )/(2*div_fact)
        z_old = z_new
    
    ## Estimating r
    r = np.dot(np.linalg.inv(np.dot(UU, np.transpose(UU))), 
                                                   np.dot(UU,np.transpose([Q2]) ) )
    v = np.sign(r)
    s = np.sign(v*np.transpose([z_new]))

    p = 1 ## p + 1 is degree of homogenity

    w = []
    for ind in range(k):
        
        w.append(s[ind]*np.math.pow(abs(z_new[ind]/np.math.pow( np.dot( [alpha],
                    np.transpose([Vu[ind]]) ) ,l1-1)), 1.0/(p+1))*Vu[ind])
    w = np.asarray(w)

    return v,w

In [ ]:
def tensorInit(X, y):
    divInd = int(len(X)/3)
    
    # Partition
    X1 = X[:divInd]
    y1 = y[:divInd]

    X2 = X[divInd:2*divInd]
    y2 = y[divInd:2*divInd]
    
    X3 = X[2*divInd:]
    y3 = y[2*divInd:]
    
    ## P2 
    ## Estimating P2 as M2 :: TODO
    
#     alpha = np.random.normal(0.0, 0.1 , d)
#     P2 = getM2(X1,y1)
#     print(np.linalg.norm(P2))

    ## Power Method
    V = powMeth(k, X1, y1)
    
    ## R3
    R3 = multLnr2(getM3(X2, y2), V)
    
    #### KCL
    U = R3[0]

    ## RecMagSign
    return recmagsgn(V, U, X3, y3) 

In [ ]:
T = tensorInit(train_x, train_y)

In [ ]:
# T2 = sktensor.dtensor(T)
# P, fit, itr, exectimes = sktensor.cp_als(T2, 5, init='random')

In [ ]:
def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    """
    h    = tf.square(tf.nn.relu((tf.matmul(X, w_1))))
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

In [ ]:
X = tf.placeholder("float", shape=[None, d])
y = tf.placeholder("float", shape=[None, 1])

# Weight initializations
w_1 = init_weights((d, k))
w_2 = init_weights((k, 1))

# Forward propagation
yhat  = forwardprop(X, w_1, w_2)

# Backward propagation
cost = tf.losses.mean_squared_error(y, yhat)
updates = tf.train.GradientDescentOptimizer(lr).minimize(cost)

# Run SGD
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
init = tf.global_variables_initializer()
sess.run(init)
epsilon = 1e-4
for epoch in range(T):
    # Train with each example
    i = 0
    for iter in range(int(n/batch_size)):
        sess.run(updates, feed_dict={X: train_x[i: i + batch_size], y: train_y[i: i + batch_size].reshape(batch_size,1)})
        i = (i + batch_size)%n
    train_accuracy = np.mean((train_y - sess.run(yhat, feed_dict={X:train_x, y:train_y.reshape(n,1)})) <= epsilon)
    print("Epoch = ", epoch+1," train Acc: ",100.*train_accuracy )